# Inputs

### Librerías necesarias

In [1]:
%reset_selective -f b

# Este comando le indica a Python que las figuras se deben generar dentro de la misma Notebook, no en una ventana
## Solo para notebooks.
%matplotlib inline

from matplotlib import pyplot as plt
from matplotlib import colors
import numpy as np
import glob
import pyart
import scipy.io as sio
import re


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



### Levanto los datos

In [2]:
dia = '20170203'

#### Levanto los datos de radar

In [3]:
path_user_radar = './febdBZ/'+ dia + '/'
FileList_radar = np.sort(glob.glob(path_user_radar+'*.nc'))
file2read_radar =FileList_radar[0]
print(file2read_radar)

#Creamos el objeto "radar"
radar = pyart.io.read(file2read_radar)


./febdBZ/20170203/cfrad.20170203_000004.000_to_20170203_000423.001_INTA_Ang_v263_SUR.nc




#### Levanto los datos de rayos


In [4]:
path_user_rayos = './rayos/201702/'+ dia

with open(path_user_rayos+'.txt') as f:
    lines = f.readlines()

#### Armo las celdas

Las grillas las genero en el radar, luego correlaciono la información de rayos.

In [5]:
nelev=0

start_index = radar.sweep_start_ray_index['data'][nelev]
end_index   = radar.sweep_end_ray_index['data'][nelev]

rango  = radar.range['data']
tiempo = radar.time['data'][start_index:end_index]

# Sitio radar
lat_radar = radar.latitude['data'][0]
lon_radar = radar.longitude['data'][0]

# Datos de lat/lon
lats = radar.gate_latitude['data'][start_index:end_index]
lons = radar.gate_longitude['data'][start_index:end_index]

# Datos x/y (en metros)
xdist = radar.gate_x['data'][start_index:end_index]/1000.
ydist = radar.gate_y['data'][start_index:end_index]/1000.

# Variables radar
Zh  = radar.fields['dBZ']['data'][start_index:end_index, :]

In [6]:
iTime = file2read_radar.split('_')[1]
fTime = file2read_radar.split('_')[4]

In [7]:
print(iTime,fTime)

000004.000 000423.001


Genero la grilla

In [8]:
bins = 240

In [9]:
# mask out last 10 gates of each ray, this removes the "ring" around th radar.
radar.fields['dBZ']['data'][:, -10:] = np.ma.masked

# exclude masked gates from the gridding
gatefilter = pyart.filters.GateFilter(radar)
gatefilter.exclude_transition()
gatefilter.exclude_masked('dBZ')

In [10]:
# perform Cartesian mapping, limit to the reflectivity field.
grid = pyart.map.grid_from_radars(
    (radar,), gatefilters=(gatefilter, ),
    grid_shape=(1, bins, bins),
    grid_limits=((0, 0), (-bins*1000, bins*1000), (-bins*1000, bins*1000)),
    fields=['dBZ'])

In [11]:
lat_grid=grid.get_point_longitude_latitude(level=0, edges=False)[1]
lon_grid=grid.get_point_longitude_latitude(level=0, edges=False)[0]
extLat = [np.amin(lat_grid),np.amax(lat_grid)]
difLat = bins/(extLat[1]-extLat[0])
extLon = [np.amin(lon_grid),np.amax(lon_grid)]
difLon = bins/(extLon[1]-extLon[0])
print(extLat)
print(extLon)

[-38.698056318536423, -34.352495282538385]
[-66.754157243330312, -61.225976756669681]


Levanto los datos de rayos

In [12]:
tot = len(lines)

latitude = []
longitude = []
hora = []
intensity = []
latBin = []
lonBin = []
for i in range(tot):
    foo=lines[i].split()
    t=(foo[3]+foo[4]+foo[5])
   
    type(foo[7])
    if (t<fTime) and (t>iTime) :
        
        if(float(foo[8])>extLat[0]) and (float(foo[8])<extLat[1]):
            
            
            if(float(foo[7])>extLon[0]) and (float(foo[7])<extLon[1]):
                
                latitude.append(float(foo[8]))
                latBin.append(int((float(foo[8])-extLat[0])*difLat))
                lonBin.append(int((float(foo[7])-extLon[0])*difLon))
                longitude.append(float(foo[7]))
                hora.append(t)
                intensity.append(float(foo[9]))
           
    
Rayos = {'Lat': latitude, 'Long': longitude, 'Time': hora, 'Int': intensity, 'x':lonBin, 'y':latBin}


In [13]:

indices = np.zeros((bins,bins), dtype=object)
time = np.zeros((bins,bins), dtype=object)
intensidad = np.zeros((bins,bins), dtype=object)
FR = np.zeros((bins,bins), dtype=object)

i=0
j=0

for l in range(len(Rayos['x'])):
    i=Rayos['x'][l]
    j=Rayos['y'][l]
    indices[i,j]=np.append(indices[i,j],l)
    intensidad[i,j]=np.append(intensidad[i,j],Rayos['Int'][l])
    time[i,j]=np.append(time[i,j],Rayos['Time'][l])
    
for i in range(bins):
    for j in range(bins):
        if not isinstance(indices[i,j], int):
            indices[i,j] = np.delete(indices[i,j],0)
            intensidad[i,j] = np.delete(intensidad[i,j],0)
            time[i,j] = np.delete(time[i,j],0)
            FR[i,j] = np.size(indices[i,j])
            
            if FR[i,j]>1:
                tiempo=[]
                for ind in indices[i,j]:
                    
                    tiempo.append(Rayos['Time'][ind])
                    
                    
                dif=float(max(tiempo))-float(min(tiempo))
                if dif != 0:
                    #print(FR[i,j])
                    #print(float(max(tiempo)),float(min(tiempo)))
                    FR[i,j]=FR[i,j]/dif
                else:
                    FR[i,j]=1
            
                
                
Rayos = {'Lat': latitude, 'Long': longitude, 'Time': time, 'Int': intensity, 'x':lonBin, 'y':latBin, 
         'Ind': indices, 'FR':FR}
Campos = {'Ind': indices, 'Intensidad': intensidad, 'Tiempo':time, 'FR': FR}

In [14]:
Campos['Tiempo']


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [15]:
Rayos['Time']

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)